# Chapter 1: What is Systematic Trading?

## Learning Objectives

By the end of this chapter, you'll understand:
- The difference between manual and systematic trading
- Why systematic trading works
- Robert Carver's key principles
- What this repository demonstrates

---

## The Gambling Problem

Imagine you're at a casino playing blackjack. You have a system:
- "I'll bet $10 when I feel confident"
- "I'll bet $50 when I'm really sure"
- "I'll stop when I'm tired"

**This is how most people trade stocks.**

The problems?
1. **"Feeling confident"** - Emotions change based on recent wins/losses
2. **"Really sure"** - Humans are terrible at judging probability
3. **"When I'm tired"** - You make worse decisions when emotional or exhausted

### Interactive Example: Coin Flip Simulation

Let's simulate two approaches to a simple coin flip game:
- **Emotional trader**: Varies bet size based on recent results
- **Systematic trader**: Always bets the same amount

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Simulation parameters
n_flips = 100
win_probability = 0.55  # Slight edge (like a good trading strategy)
initial_capital = 10000

def emotional_trader(n_flips, win_prob, initial_capital):
    """Simulates an emotional trader who varies bet size based on recent results"""
    capital = initial_capital
    equity_curve = [capital]
    
    base_bet = 100
    recent_wins = 0  # Track recent performance
    
    for i in range(n_flips):
        # Vary bet size based on recent wins
        if recent_wins >= 3:
            bet = base_bet * 2  # "I'm on a hot streak!"
        elif recent_wins <= -3:
            bet = base_bet * 0.5  # "I need to be cautious"
        else:
            bet = base_bet
        
        # Flip coin
        win = np.random.random() < win_prob
        
        if win:
            capital += bet
            recent_wins += 1
        else:
            capital -= bet
            recent_wins -= 1
        
        # Reset counter if too extreme
        if abs(recent_wins) > 5:
            recent_wins = 0
        
        equity_curve.append(capital)
    
    return equity_curve

def systematic_trader(n_flips, win_prob, initial_capital):
    """Simulates a systematic trader with fixed bet size"""
    capital = initial_capital
    equity_curve = [capital]
    
    bet = 100  # Always the same
    
    for i in range(n_flips):
        win = np.random.random() < win_prob
        
        if win:
            capital += bet
        else:
            capital -= bet
        
        equity_curve.append(capital)
    
    return equity_curve

# Run simulations
emotional_equity = emotional_trader(n_flips, win_probability, initial_capital)
systematic_equity = systematic_trader(n_flips, win_probability, initial_capital)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(emotional_equity, label='Emotional Trader', alpha=0.7, linewidth=2)
plt.plot(systematic_equity, label='Systematic Trader', alpha=0.7, linewidth=2)
plt.axhline(y=initial_capital, color='gray', linestyle='--', label='Starting Capital')
plt.xlabel('Number of Flips')
plt.ylabel('Capital ($)')
plt.title('Emotional vs Systematic Trading: Coin Flip Game (55% Win Rate)')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Summary statistics
print(f"\nFinal Results after {n_flips} flips:")
print(f"Emotional Trader: ${emotional_equity[-1]:,.2f} ({(emotional_equity[-1]/initial_capital - 1)*100:+.1f}%)")
print(f"Systematic Trader: ${systematic_equity[-1]:,.2f} ({(systematic_equity[-1]/initial_capital - 1)*100:+.1f}%)")
print(f"\nExpected profit: ${n_flips * 100 * (win_probability - 0.5) * 2:,.2f}")

```{admonition} Key Insight
:class: tip
Notice how the emotional trader's equity curve is more volatile? By varying bet sizes based on recent results (which are mostly random), they introduced unnecessary risk. The systematic trader, despite using a simple fixed bet, achieved more consistent results.
```

## What is Systematic Trading?

**Systematic Trading** is a method where:

1. **Rules are defined in advance** - You decide exactly what conditions trigger a trade
2. **Rules are followed mechanically** - No emotional overrides
3. **Everything is measurable** - You can test if your rules actually work
4. **Risk is managed mathematically** - Position sizes calculated by formula, not gut feel

### Real-World Comparison

| Manual Trader | Systematic Trader |
|---------------|-------------------|
| Watches news about Tesla | Has a rule: "Buy when 16-day avg > 64-day avg" |
| Feels optimistic after Elon's tweet | Computer calculates this daily |
| Buys $10,000 worth ("feels right") | Buys calculated position size based on volatility |
| Gets nervous when price drops 5% | No emotions - follows rules |
| Sells at a loss | Holds until signal reverses |
| Repeats emotional cycle | Consistent execution every time |

## Why Does Systematic Trading Work?

### 1. Consistency

Humans are inconsistent. You might be brave on Monday and scared on Friday. A computer applies the same logic every time.

In [ ]:
# Demonstration: Human inconsistency in decision-making
import random

def human_decision(market_condition, mood):
    """Simulates inconsistent human decision making"""
    if mood == 'confident':
        threshold = 0.3  # Lower bar for buying
    elif mood == 'scared':
        threshold = 0.8  # Higher bar for buying
    else:
        threshold = 0.5
    
    return market_condition > threshold

def systematic_decision(market_condition):
    """Consistent systematic decision"""
    return market_condition > 0.5  # Always the same threshold

# Simulate 20 decision points with the same market conditions
market_conditions = [0.6] * 20  # Same condition repeated
moods = random.choices(['confident', 'neutral', 'scared'], k=20)

human_decisions = [human_decision(mc, mood) for mc, mood in zip(market_conditions, moods)]
systematic_decisions = [systematic_decision(mc) for mc in market_conditions]

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

ax1.scatter(range(20), human_decisions, c=['green' if d else 'red' for d in human_decisions], 
            s=100, alpha=0.6)
ax1.set_ylim(-0.5, 1.5)
ax1.set_yticks([0, 1])
ax1.set_yticklabels(['Sell', 'Buy'])
ax1.set_xlabel('Decision Number')
ax1.set_title('Human Trader (Same Market Condition)')
ax1.grid(alpha=0.3)

ax2.scatter(range(20), systematic_decisions, c=['green' if d else 'red' for d in systematic_decisions],
            s=100, alpha=0.6)
ax2.set_ylim(-0.5, 1.5)
ax2.set_yticks([0, 1])
ax2.set_yticklabels(['Sell', 'Buy'])
ax2.set_xlabel('Decision Number')
ax2.set_title('Systematic Trader (Same Market Condition)')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Human trader made {sum(human_decisions)}/{len(human_decisions)} buy decisions")
print(f"Systematic trader made {sum(systematic_decisions)}/{len(systematic_decisions)} buy decisions")
print(f"\nInconsistency: Human varied despite identical market conditions!")

### 2. Backtesting

You can test your rules on historical data. Did your strategy make money over the last 10 years? You can actually find out before risking real money.

Let's preview what backtesting looks like (we'll dive deep into this in Chapter 10):

In [ ]:
# Simple backtest preview - we'll build the full system later!
# For now, let's just show the concept with synthetic data

# Generate synthetic price data
np.random.seed(42)
days = 252 * 2  # 2 years of trading days
price = 100
prices = [price]

for _ in range(days - 1):
    # Random walk with slight upward drift
    change = np.random.normal(0.001, 0.02)  # 0.1% drift, 2% volatility
    price = price * (1 + change)
    prices.append(price)

prices = pd.Series(prices)

# Simple moving average strategy
fast_ma = prices.rolling(window=20).mean()
slow_ma = prices.rolling(window=50).mean()

# Generate signals
signal = (fast_ma > slow_ma).astype(int)  # 1 = buy, 0 = sell

# Calculate returns
returns = prices.pct_change()
strategy_returns = signal.shift(1) * returns  # Shift to avoid lookahead bias

# Calculate cumulative returns
buy_hold = (1 + returns).cumprod()
strategy = (1 + strategy_returns).cumprod()

# Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Price and moving averages
ax1.plot(prices.values, label='Price', linewidth=2)
ax1.plot(fast_ma.values, label='Fast MA (20 days)', alpha=0.7)
ax1.plot(slow_ma.values, label='Slow MA (50 days)', alpha=0.7)
ax1.fill_between(range(len(signal)), 0, prices.max(), 
                 where=signal==1, alpha=0.1, color='green', label='Long Position')
ax1.set_xlabel('Trading Days')
ax1.set_ylabel('Price ($)')
ax1.set_title('Simple Moving Average Strategy')
ax1.legend()
ax1.grid(alpha=0.3)

# Performance comparison
ax2.plot(buy_hold.values, label='Buy & Hold', linewidth=2)
ax2.plot(strategy.values, label='MA Strategy', linewidth=2)
ax2.set_xlabel('Trading Days')
ax2.set_ylabel('Cumulative Return (1 = 100%)')
ax2.set_title('Strategy Performance vs Buy & Hold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Buy & Hold Return: {(buy_hold.iloc[-1] - 1)*100:.2f}%")
print(f"Strategy Return: {(strategy.iloc[-1] - 1)*100:.2f}%")
print(f"\nNote: This is a simple example. Real backtesting includes costs, slippage, etc.")

```{admonition} Important
:class: warning
This is a simplified example! Real backtesting must account for:
- Transaction costs (commissions, spreads)
- Slippage (price movement during order execution)
- Market impact (your orders affecting price)
- Realistic position sizing

We'll cover all of this properly in later chapters!
```

## Who is Robert Carver?

**Robert Carver** is a former portfolio manager who worked at AHL, a multi-billion dollar quantitative hedge fund. In 2015, he wrote *"Systematic Trading: A Unique New Method for Designing Trading and Investing Systems"*.

### Carver's Key Innovations

#### 1. Volatility Targeting

**Problem:** Some stocks move 1% per day, others 10% per day. Equal dollar investments = unequal risk.

**Solution:** Adjust position size based on volatility to take consistent risk.

In [ ]:
# Demonstration: Volatility Targeting

# Three stocks with different volatilities
stocks = {
    'Utility (Low Vol)': {'price': 100, 'daily_vol': 0.005},  # 0.5% daily
    'Tech (Medium Vol)': {'price': 100, 'daily_vol': 0.020},  # 2% daily
    'Biotech (High Vol)': {'price': 100, 'daily_vol': 0.050}, # 5% daily
}

capital = 100000
target_vol = 0.20  # 20% annual volatility target

# Equal dollar allocation (WRONG)
equal_allocation = capital / 3

# Volatility-based allocation (RIGHT)
results = []

for name, data in stocks.items():
    # Equal dollar risk
    shares_equal = equal_allocation / data['price']
    risk_equal = equal_allocation * data['daily_vol'] * np.sqrt(252)
    
    # Volatility targeted
    annual_vol = data['daily_vol'] * np.sqrt(252)
    target_position = (capital * target_vol) / (data['price'] * annual_vol)
    allocation_vol = target_position * data['price']
    risk_vol = allocation_vol * annual_vol
    
    results.append({
        'Stock': name,
        'Equal $ Allocation': equal_allocation,
        'Equal $ Risk': risk_equal,
        'Vol-Targeted Allocation': allocation_vol,
        'Vol-Targeted Risk': risk_vol
    })

df_results = pd.DataFrame(results)

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Equal dollar allocation
x = range(len(df_results))
ax1.bar(x, df_results['Equal $ Allocation'], alpha=0.6, label='Allocation')
ax1_twin = ax1.twinx()
ax1_twin.plot(x, df_results['Equal $ Risk'], 'ro-', linewidth=2, markersize=10, label='Risk')
ax1.set_xticks(x)
ax1.set_xticklabels(df_results['Stock'], rotation=15, ha='right')
ax1.set_ylabel('Allocation ($)', color='b')
ax1_twin.set_ylabel('Annual Risk ($)', color='r')
ax1.set_title('Equal Dollar Allocation\n(UNEQUAL RISK!)')
ax1.grid(alpha=0.3)

# Volatility-targeted allocation
ax2.bar(x, df_results['Vol-Targeted Allocation'], alpha=0.6, label='Allocation')
ax2_twin = ax2.twinx()
ax2_twin.plot(x, df_results['Vol-Targeted Risk'], 'ro-', linewidth=2, markersize=10, label='Risk')
ax2.set_xticks(x)
ax2.set_xticklabels(df_results['Stock'], rotation=15, ha='right')
ax2.set_ylabel('Allocation ($)', color='b')
ax2_twin.set_ylabel('Annual Risk ($)', color='r')
ax2.set_title('Volatility-Targeted Allocation\n(EQUAL RISK!)')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\nEqual Dollar Allocation:")
print(df_results[['Stock', 'Equal $ Allocation', 'Equal $ Risk']].to_string(index=False))
print(f"\nRisk range: ${df_results['Equal $ Risk'].min():.0f} to ${df_results['Equal $ Risk'].max():.0f}")
print(f"Risk ratio: {df_results['Equal $ Risk'].max() / df_results['Equal $ Risk'].min():.1f}x\n")

print("\nVolatility-Targeted Allocation:")
print(df_results[['Stock', 'Vol-Targeted Allocation', 'Vol-Targeted Risk']].to_string(index=False))
print(f"\nAll positions have equal risk: ~${df_results['Vol-Targeted Risk'].mean():.0f}")

#### 2. Forecast Scaling

**Problem:** Different strategies produce signals on different scales. How do you compare them?

**Solution:** Standardize all signals to -20 to +20 range, with average absolute value of 10.

We'll cover this in detail in Chapter 4.

#### 3. Cost Awareness

**Problem:** Most backtests ignore costs. A strategy that trades 100 times/year might be unprofitable after commissions.

**Solution:** Build transaction costs into every backtest from day one.

We'll explore this thoroughly in Chapter 7.

#### 4. Diversification

**Problem:** Single strategy + single stock = high risk

**Solution:** Combine multiple strategies, timeframes, and instruments.

## What This Repository Demonstrates

This codebase implements Carver's principles for US stock markets (NASDAQ/NYSE stocks like Google, Microsoft, Tesla).

### Repository Structure

In [ ]:
# Let's explore the repository structure
import os
from pathlib import Path

# Go up two levels from book directory to repo root
repo_root = Path.cwd().parent.parent if 'book' in str(Path.cwd()) else Path.cwd().parent

def show_tree(directory, prefix="", max_depth=3, current_depth=0):
    """Display directory tree"""
    if current_depth >= max_depth:
        return
    
    try:
        contents = sorted(directory.iterdir())
    except PermissionError:
        return
    
    # Filter out unwanted directories
    ignore = {'.git', '__pycache__', '.pytest_cache', 'logs', '.venv', 'venv', '_build'}
    contents = [c for c in contents if c.name not in ignore and not c.name.startswith('.')]
    
    for i, path in enumerate(contents):
        is_last = i == len(contents) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{path.name}")
        
        if path.is_dir():
            extension = "    " if is_last else "│   "
            show_tree(path, prefix + extension, max_depth, current_depth + 1)

print("Repository Structure:")
print(repo_root.name)
show_tree(repo_root, max_depth=2)

### Key Components

1. **Data Management** (`data/data_manager.py`)
   - Downloads historical stock prices from Yahoo Finance
   - Stores data locally for analysis

2. **Trading Strategies** (`strategy/trend_following.py`)
   - EWMAC: Exponentially Weighted Moving Average Crossover
   - MA Crossover: Simple Moving Average Crossover
   - Multiple EWMAC: Combining different timeframes

3. **Risk Management** (`risk_management/position_sizer.py`)
   - Volatility targeting
   - Position size calculations
   - Maximum position limits

4. **Backtesting** (`backtesting/backtest_engine.py`)
   - Simulates trading with historical data
   - Includes transaction costs and slippage
   - Tracks equity curve and performance

5. **Performance Analysis** (`backtesting/performance.py`)
   - Calculates returns, Sharpe ratio, drawdowns
   - Generates charts and reports

## A Preview: Running the System

Here's a glimpse of how the complete system works (we'll build this step-by-step in later chapters):

In [ ]:
# Note: This is a preview only - we'll implement each piece in detail later!
# For now, this shows the high-level workflow

print("""
Systematic Trading Workflow:
===========================

Step 1: Download Data
  ↓
  from data.data_manager import DataManager
  dm = DataManager()
  data = dm.download_stock_data('GOOG', save=True)

Step 2: Create Strategy
  ↓
  from strategy.trend_following import EWMAC
  strategy = EWMAC(fast_span=16, slow_span=64)
  signals = strategy.generate_signals(data)

Step 3: Size Positions
  ↓
  from risk_management.position_sizer import PositionSizer
  sizer = PositionSizer(capital=100000, volatility_target=0.20)
  position = sizer.calculate_position(signal, price, volatility)

Step 4: Run Backtest
  ↓
  from backtesting.backtest_engine import BacktestEngine
  engine = BacktestEngine(initial_capital=100000)
  results = engine.run(strategy, data, sizer)

Step 5: Analyze Performance
  ↓
  from backtesting.performance import PerformanceAnalyzer
  analyzer = PerformanceAnalyzer(results)
  analyzer.print_summary()
  analyzer.plot_equity_curve()

Each of these steps will be covered in detail in upcoming chapters!
""")

## Key Takeaways

```{admonition} Summary
:class: note

1. **Systematic trading uses rules, not emotions**
   - Decisions made in advance
   - Executed mechanically
   - Measurable and testable

2. **It works because of:**
   - Consistency (no emotional variation)
   - Backtesting (test before risking money)
   - Proper risk management (mathematical position sizing)

3. **Robert Carver's principles:**
   - Volatility targeting for consistent risk
   - Forecast scaling for strategy combination
   - Cost awareness prevents over-trading
   - Diversification smooths returns

4. **This repository provides:**
   - Complete implementation of Carver's approach
   - Real market data integration
   - Realistic backtesting with costs
   - Production-ready code
```

## Exercises

1. **Modify the coin flip simulation**: Change the win probability to 0.48 (losing edge). What happens to both traders?

2. **Experiment with the consistency demo**: What happens if the human is always confident? Always scared?

3. **Think critically**: Can you identify a situation where emotional trading might outperform systematic trading? (Hint: think about rare events)

4. **Explore the code**: Browse the repository structure. Which file do you think contains the core strategy logic?

## Next Steps

In [Chapter 2: Understanding Stock Markets](02_understanding_stock_markets.ipynb), we'll learn:
- What stocks actually are
- How prices are determined
- Reading OHLCV data
- Why trends exist

This foundation is essential before we dive into strategy building!